<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/cookbooks/upstage/Solar-Full-Stack LLM-101/05_3_OracleDB.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Retrieval Augmented Generation (RAG) Baseline
## Overview  
In this time, we will check the baseline code.
The goal of this project is to provide students with hands-on experience in handling and enhancing Large Language Models (LLMs) provided by [**Upstage**](https://www.upstage.ai) (Solar).

You can use any engineering method for improving benchmark performance excluding direct training (Fine-tuning).

*Collecting data directly related to the test set is considered cheating e.g., using MMLU-pro dataset or EWHA.pdf for KB*

# Baseline

In [ ]:
#!pip3 install -qU python-dotenv PyPDF2 langchain langchain-community langchain-core langchain-text-splitters langchain_upstage oracledb python-dotenv

In [ ]:
# Additional Contents : https://wikidocs.net/253106 

In [65]:
# set parameters

file = open("info/api.txt", "r")
api_key = file.read()
file.close()
file = open("info/path.txt", "r")
data_path = file.read()
file.close()

In [66]:
from langchain_upstage import UpstageEmbeddings

# 쿼리 전용 임베딩 모델
query_embeddings = UpstageEmbeddings(api_key=api_key, model="solar-embedding-1-large-query")

# 문장 전용 임베딩 모델
passage_embeddings = UpstageEmbeddings(api_key=api_key, model="solar-embedding-1-large-passage")

## 1. build DB

In [67]:
from langchain_upstage import UpstageLayoutAnalysisLoader
import os

UPSTAGE_API_KEY = api_key

layzer = UpstageLayoutAnalysisLoader(api_key=UPSTAGE_API_KEY,file_path=os.path.join(data_path, 'ewha.pdf'), output_type="text")

docs = layzer.load()  # or layzer.lazy_load()

/home/aix7101/anaconda3/envs/yoonnlp/lib/python3.10/site-packages/langchain_upstage/layout_analysis.py:135: UserWarning: UpstageLayoutAnalysisLoader is deprecated.Please use langchain_upstage.document_parse.UpstageDocumentParseLoader instead.
  warnings.warn(
/home/aix7101/anaconda3/envs/yoonnlp/lib/python3.10/site-packages/langchain_upstage/layout_analysis_parsers.py:160: UserWarning: UpstageLayoutAnalysisParser is deprecated.Please use langchain_upstage.document_parse_parsers.UpstageDocumentParseParser instead.
  warnings.warn(


## 2. Text Split

In [68]:
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)

# 2. Split
text_splitter = RecursiveCharacterTextSplitter.from_language(
    chunk_size=1000, chunk_overlap=300, language=Language.HTML
)
splits = text_splitter.split_documents(docs)
print("Splits:", len(splits))

Splits: 68


In [69]:
print(splits[0])
len(splits[0].page_content)

page_content='이화여자대학교 학칙1946. 8. 15. 제정
2017. 8. 16. 개정제1장 총칙제1조(목적) 본교는 대한민국의 교육이념과 기독교정신을 바탕으로 하여 학술의 깊은 이론과
그 광범하고 정밀한 응용방법을 교수․연구하며, 인격을 도야하여 국가와 인류사회의 발전에
공헌할 수 있는 지도여성을 양성함을 목적으로 한다.제2조(명칭) 본교는 이화여자대학교라 부른다.
제3조(위치) 본교는 서울특별시 서대문구 이화여대길 52에 둔다. (개정 2013.2.25.)제2장 편제제4조(대학 및 대학원) ① 본교에는 다음 각 호의 대학을 둔다.1. 인문과학대학, 사회과학대학, 자연과학대학, 엘텍공과대학, 음악대학, 조형예술대학, 사범
대학, 경영대학, 신산업융합대학, 의과대학, 간호대학, 약학대학, 스크랜튼대학(이하 “각
대학”이라 한다) (개정 2016.6.16.)
2. 호크마(HOKMA)교양대학
② 본교에는 대학원, 국제대학원, 통역번역대학원, 경영전문대학원, 법학전문대학원, 교육대
학원, 디자인대학원, 사회복지대학원, 신학대학원, 정책과학대학원, 공연예술대학원, 임상보
건융합대학원, 임상치의학대학원, 외국어교육특수대학원을 둔다(이하 “각 대학원”이라 한다).
(개정 2016.6.16., 2017.5.15.)
[전문개정 2015.11.27.]제5조(학부․학과․전공 및 정원) ① 각 대학, 학부, 학과, 전공 및 모집단위별 입학정원은 별표
1과 같다. (개정 2015.5.8., 2016.2.16., 2016.2.26., 2016.5.19., 2017.5.4., 2017.5.15.)② 모집단위별 입학정원의 일부는 입학전형에 따라 2개 이상의 모집단위를 통합하여 모집
할 수 있다. (개정 1999.2.9., 2017.5.15.)
③ 제2항에 따라 통합된 모집단위로 입학한 학생과 대학 또는 학부 등 광역화된 모집단위
로 입학한 학생에 대하여는 일정한 학기와 학점을 이수한 후에 총장의 승인을 얻어 이수할
전공을 결정하게 하되 이에 필요한 사항은 총장이 따로 정한다

1000

## 3. query-context matching

In [70]:
# read samples.csv file
import pandas as pd

def read_data(data_path):
    data = pd.read_csv(data_path)
    prompts = data['prompts']
    answers = data['answers']
    # returns two lists: prompts and answers
    return prompts, answers

In [71]:
prompts, answers = read_data(os.path.join(data_path, 'test_own_ewha.csv'))

In [72]:
testdata = pd.read_csv(data_path+'test_own_ewha.csv')
testdata

,prompts,answers
0,QUESTION1) 영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한...,(D)
1,QUESTION2) 각 대학에 따른 학위의 종류로 해당하지 않는 것은 무엇인가?\n...,(E)
2,QUESTION3) 부칙에 학과명 변경 및 폐과된 학과는 무엇에 의해 언제까지 존속...,(B)
3,QUESTION4) 학생 포털에서 본인의 성적을 조회할 수 있는 시기는 언제입니까?...,(B)
4,QUESTION5) 학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하...,(C)
5,QUESTION6) 학칙 제3조에서 명시된 본교의 주소에 포함되지 않는 요소는 무엇...,(D)
6,QUESTION7) 학칙 제12조(휴업일)에서 임시휴업을 결정할 수 있는 근거로 인...,(C)
7,QUESTION8) 학칙 제23조(수업연한)에 따르면 건축학과와 약학대학의 수업연한...,(D)
8,QUESTION9) 학칙 제26조(휴학)에서 규정된 휴학의 최대 연장 가능 기간을 ...,(C)
9,QUESTION10) 학칙 제50조(졸업)에서 복수전공 이수자가 요구되는 추가 조건...,(B)


In [ ]:
nowtest = pd.DataFrame(columns=['index', 'embed_ques', 'question', 'prompts', 'answers', 'top1', 'top2', 'top3'])

for index, row in testdata.iterrows():
    #if index % 100 != 0 : continue # 일단 실험할 땐 100개 단위로 끊어서 가져옴
    q = row.prompts
    a = row.answers
    question = q.partition('(A)')[0]
    question = question.partition(') ')[2]
    q = q.partition(') ')[2]
    try : 
        embedded_query = query_embeddings.embed_query(question) # 질문만 받아와서 embedding 하기
        nowtest.loc[len(nowtest)] = {'index':index, 'embed_ques' : embedded_query, 'question' : question, 'prompts' : q, 'answers' : a, 'top1': None, 'top2': None, 'top3': None}

    except :
        print(f'pass: {index}')
        continue


In [74]:
nowtest

,index,embed_ques,question,prompts,answers,top1,top2,top3
0,0,"[-0.005329132080078125, 0.0017337799072265625,...",영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학...,영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학...,(D),None,None,None
1,1,"[-0.00460052490234375, -0.0174713134765625, -0...",각 대학에 따른 학위의 종류로 해당하지 않는 것은 무엇인가?\n,각 대학에 따른 학위의 종류로 해당하지 않는 것은 무엇인가?\n(A) 이학사\n(B...,(E),None,None,None
2,2,"[0.007404327392578125, -0.0216522216796875, 0....",부칙에 학과명 변경 및 폐과된 학과는 무엇에 의해 언제까지 존속하는가?\n,부칙에 학과명 변경 및 폐과된 학과는 무엇에 의해 언제까지 존속하는가?\n(A) 현...,(B),None,None,None
3,3,"[-0.00214385986328125, -0.039215087890625, -0....",학생 포털에서 본인의 성적을 조회할 수 있는 시기는 언제입니까?\n,학생 포털에서 본인의 성적을 조회할 수 있는 시기는 언제입니까?\n(A) 시험 직후...,(B),None,None,None
4,4,"[0.0089111328125, -0.0259552001953125, -0.0064...",학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하지 않는 것은 무엇입...,학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하지 않는 것은 무엇입...,(C),None,None,None
5,5,"[0.0016117095947265625, -0.019683837890625, -0...",학칙 제3조에서 명시된 본교의 주소에 포함되지 않는 요소는 무엇입니까?\n,학칙 제3조에서 명시된 본교의 주소에 포함되지 않는 요소는 무엇입니까?\n(A) 서...,(D),None,None,None
6,6,"[-0.018157958984375, -0.0157012939453125, -0.0...",학칙 제12조(휴업일)에서 임시휴업을 결정할 수 있는 근거로 인정되지 않는 경우는?\n,학칙 제12조(휴업일)에서 임시휴업을 결정할 수 있는 근거로 인정되지 않는 경우는?...,(C),None,None,None
7,7,"[0.0029163360595703125, -0.018707275390625, -0...",학칙 제23조(수업연한)에 따르면 건축학과와 약학대학의 수업연한이 다른 이유로 올바...,학칙 제23조(수업연한)에 따르면 건축학과와 약학대학의 수업연한이 다른 이유로 올바...,(D),None,None,None
8,8,"[-5.4895877838134766e-05, -0.01477813720703125...",학칙 제26조(휴학)에서 규정된 휴학의 최대 연장 가능 기간을 초과하는 경우가 아닌...,학칙 제26조(휴학)에서 규정된 휴학의 최대 연장 가능 기간을 초과하는 경우가 아닌...,(C),None,None,None
9,9,"[-0.002269744873046875, 0.0017795562744140625,...",학칙 제50조(졸업)에서 복수전공 이수자가 요구되는 추가 조건은 무엇입니까?\n,학칙 제50조(졸업)에서 복수전공 이수자가 요구되는 추가 조건은 무엇입니까?\n(A...,(B),None,None,None


'QUESTION3) 부칙에 학과명 변경 및 폐과된 학과는 무엇에 의해 언제까지 존속하는가?\n(A) 현 학칙에 의하여 재학생이 졸업하는 연도까지 존속한다\n(B) 구 학칙에 의하여 재학생이 졸업하는 연도까지 존속한다\n(C) 현 학칙에 의하여 입학생이 졸업하는 연도까지 존속한다\n(D) 구 학칙에 의하여 입학생이 졸업하는 연도까지 존속한다\n(E) 현 학칙에 의하여 3월 1일까지 존속한다\n(F) 규정을 적용하지 아니한다'

## 4. Prompt engineering

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage


llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    
    Please provide most correct answer from the following context.
    If the answer is not present in the context, Answer "The information is NOT present in the context." and provide your reasoning answer.
    
    The Answer format is as follow (do not explain) : 
    [Answer] (D) keyword.
    ---
    
    Question: {question}
    Context: {context}
    [Answer]
    ---
        
    '''

)
chain = prompt_template | llm




In [92]:
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage


llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    
    [Question]에 대한 정답을 출력해. 주어진 [Context]에서 관련된 정보를 찾아.
    
    정확하게 정답만 답변해.
    그리고 답변이 정확한지 한 번 더 확인해
    ---
    
    [Question] {question}
    [Context] {context}
    [Answer]
        
    '''

)
ko_chain = prompt_template | llm


# top1 = []
# for idx, row in nowtest.iterrows() : ############### 일단 지금은 100개 단위로 띄엄띄엄 test 중
#     response = ko_chain.invoke({"question": row.question}) # 선지 전까지 받아오기
#     top1.append(response.content)
#     nowtest.loc[idx, 'topic_b'] = response.content

# top1

In [93]:
import numpy as np

chunk = []
for index in range(0,len(splits)) : # context 받아오기
    chunk.append(splits[index].page_content)

responses = []
for prompt in prompts : # 질문 받아오기 
    embedded_query = query_embeddings.embed_query(prompt.split('\n')[0]) # 선지 빼고 가져오기
    embedded_documents = passage_embeddings.embed_documents(chunk)

    # 유사도 기준 내림차순 정렬
    sorted_idx = (np.array(embedded_query) @ np.array(embedded_documents).T).argsort()[::-1]

    # QA
    print("*****************************************")
    
    print(f"now prompt: {prompt}")
    print(sorted_idx[0])
    print(chunk[sorted_idx[0]])

    print("*****************************************")

    response = ko_chain.invoke({"question": prompt, "context": sorted_idx[0]})
    responses.append(response.content)


*****************************************
now prompt: QUESTION1) 영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학점은 몇점인가?
(A) 인정 안됨
(B) 1학점
(C) 2학점
(D) 3학점
21
(신설 2009.2.23.)제48조의2(영어 및 정보인증) ① 영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을취득한 경우 이를 각 3학점으로 인정하고 인증서를 교부할 수 있다. (개정 2015.9.18.)② 제1항의 시행에 관한 사항은 총장이 따로 정한다.[본조신설 2000.6.20.] [제목개정 2015.9.18.]이화여자대학교 학칙제48조의3(과정수료) ① 수업연한 이상을 등록하고 제48조의 정해진 학점을 모두 취득한 학
생에 대하여 학사학위과정의 수료를 인정한다. (개정 2015.9.18., 2017.8.16.)
② 제1항의 수료사정에 있어서는 총 평균성적이 1.70 이상이어야 하며, 기타 사항은 총장
이 따로 정한다. (신설 2015.9.18.)
[본조신설 2015.2.6.]
제48조의4(의예과 수료) ① 수업연한 이상을 등록하고 훈련학점 4학점 및 72학점 이상을 취
득한 학생에 대하여 의예과 수료를 인정한다. (개정 2017.8.16.)
② 제1항의 수료사정에 있어서는 총 평균성적이 2.00 이상이어야 하며, 기타 사항은 총장
이 따로 정한다.
[본조신설 2016.2.16.]
제49조(석사학위과정 교과목수강 특례) ① 성적이 우수한 학생은 3학년 이상 수료한 후부터
대학장의 추천과 대학원장의 승인을 얻어 학기당 취득기준학점수의 범위 내에서 대학원 석
사학위과정 교과목을 수강할 수 있다. 이에 관하여 필요한 사항은 총장이 따로 정한다.
(개정 2001.9.24)
② 제1항의 규정에 의하여 수강할 수 있는 석사학위과정의 교과목은 총 12학점을 초과할
수 없다. (개정 2001.9.24)
③ 제2항의 규정에 의하여 취득한 학점 중 6학점까지는 학사학위과정 졸업 또는 수료

In [94]:
for i in responses:
    print(i)
    print('-')


(C) 2학점
-
H
-
(B) 구 학칙에 의하여 재학생이 졸업하는 연도까지 존속한다
-
B. 성적 입력 마감 후
-
(D) 학술의 응용방법 연구
-
정답은 (D) 강남구입니다.
-
D) 대규모 학내 행사
-
(A) 건축학과는 추가 설계 실습을 포함하므로 더 긴 연한이 필요하다.
-
D) 해외 봉사로 인한 휴학: 3년
-
(B) 전공별 종합시험 합격
-
(D) 교과목 담당교수의 허가 없이 수강 취소
-
(C) 독립된 학과로 운영된다.
-
(D) 타교에 입학한 경우
-
(E) 종강일입니다.
-
(A) 신입생 첫 학기
-
B) 12시간 이상
-
[Answer] (E) 특정 종교 교리 확립
-
(B) 통합된 모집단위는 대학장이 결정한다.

이유:
학칙 제5조(입학정원)에 따르면, 모집단위별 통합 모집이 가능한 조건은 다음과 같습니다: 입학정원의 일부를 통합 모집할 수 있고, 전공 결정은 일정 학점 이수 후 이루어지며, 총장이 세부 사항을 따로 정할 수 있습니다. 그러나 통합된 모집단위는 대학장이 결정한다는 내용은 학칙에 언급되지 않았습니다. 따라서 (B)는 옳지 않은 조건입니다.
-
(C) 약학대학
-
답변: (B) 20학점

근거: 주어진 문맥에서 관련 정보는 "학칙 제45조(취득학점)에 따라 직전 학기 평균성적 3.75 이상을 받은 학생이 한 학기에 취득할 수 있는 최대 학점은 20학점입니다."입니다. 이 정보를 바탕으로 주어진 선택지 중에서 (B) 20학점이 정답입니다.
-
학칙 제28조(제적)에 따라 다음 중 제적 사유에 포함되지 않는 것은 무엇인가요?

(A) 휴학 기간 종료 후 3주 이내 복학하지 않은 경우
(B) 등록금을 납부하지 않은 경우
(C) 학기 중간에 전과를 신청한 경우
(D) 연속 3회의 학사 경고를 받은 경우
(E) 재학 연한 초과

정답은:

(C) 학기 중간에 전과를 신청한 경우
-
[Answer] (D) 주말
-
(B) 제1전공의 졸업요건 학점을 충족해야 한다.
-


## groundedness check

In [ ]:
import os
from langchain_upstage import UpstageGroundednessCheck # langchain_upstage==0.1.3
 
groundedness_check = UpstageGroundednessCheck()
 
request_input = {
    "context": f"Mauna Kea is an inactive volcano on the island of Hawai'i. Its peak is 4,207.3 m above sea level, making it the highest point in Hawaii and second-highest peak of an island on Earth.",
    "answer": "Mauna Kea is 5,207.3 meters tall.",
}
response = groundedness_check.invoke(request_input)
print(response)

## Check Accuracy

In [95]:
# funcion to extract an answer from response

import re

def extract_answer(response):
    """
    extracts the answer from the response using a regular expression.
    expected format: "[ANSWER]: (A) convolutional networks"

    if there are any answers formatted like the format, it returns None.
    """
    pattern = r"\[ANSWER\]:\s*\((A|B|C|D|E)\)"  # Regular expression to capture the answer letter and text
    match = re.search(pattern, response)

    if match:
        return match.group(1) # Extract the letter inside parentheses (e.g., A)
    else:
        return extract_again(response)

def extract_again(response):
    pattern = r"\b[A-J]\b(?!.*\b[A-J]\b)"
    match = re.search(pattern, response)
    if match:
        return match.group(0)
    else:
        return None

In [96]:
# print accuracy

cnt = 0
wrong = []
for idx, (answer, response) in enumerate(zip(answers, responses)):
    print("-"*10)
    generated_answer = extract_answer(response)
    print(response)
    # check
    if generated_answer:
        print(f"idx: {idx} | generated answer: {generated_answer}, answer: {answer}")
    else:
        print("extraction fail")

    if generated_answer in answer:
        cnt += 1
    else : 
        wrong.append(idx)
        if generated_answer == None:
            continue

print()
print(f"acc: {(cnt/len(answers))*100}%")

----------
(C) 2학점
idx: 0 | generated answer: C, answer: (D)
----------
H
idx: 1 | generated answer: H, answer: (E)
----------
(B) 구 학칙에 의하여 재학생이 졸업하는 연도까지 존속한다
idx: 2 | generated answer: B, answer: (B)
----------
B. 성적 입력 마감 후
idx: 3 | generated answer: B, answer: (B)
----------
(D) 학술의 응용방법 연구
idx: 4 | generated answer: D, answer: (C)
----------
정답은 (D) 강남구입니다.
idx: 5 | generated answer: D, answer:  (D)
----------
D) 대규모 학내 행사
idx: 6 | generated answer: D, answer:  (C)
----------
(A) 건축학과는 추가 설계 실습을 포함하므로 더 긴 연한이 필요하다.
idx: 7 | generated answer: A, answer:  (D)
----------
D) 해외 봉사로 인한 휴학: 3년
idx: 8 | generated answer: D, answer:  (C)
----------
(B) 전공별 종합시험 합격
idx: 9 | generated answer: B, answer:  (B)
----------
(D) 교과목 담당교수의 허가 없이 수강 취소
idx: 10 | generated answer: D, answer:  (B)
----------
(C) 독립된 학과로 운영된다.
idx: 11 | generated answer: C, answer:  (C)
----------
(D) 타교에 입학한 경우
idx: 12 | generated answer: D, answer:  (B)
----------
(E) 종강일입니다.
idx: 13 | generated answer: E, answer: 

In [97]:
wrong

[0, 1, 4, 6, 7, 8, 10, 12, 15, 19, 20]